In [24]:
# importing the libraries
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import numpy as np

# Reading the files

In [25]:
food_data = pd.read_csv('Reviews.csv')

In [26]:
food_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
Id                        568454 non-null int64
ProductId                 568454 non-null object
UserId                    568454 non-null object
ProfileName               568438 non-null object
HelpfulnessNumerator      568454 non-null int64
HelpfulnessDenominator    568454 non-null int64
Score                     568454 non-null int64
Time                      568454 non-null int64
Summary                   568427 non-null object
Text                      568454 non-null object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [27]:
food_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [28]:
food_data = food_data[['Score','Text']]

In [29]:
food_data.Score.value_counts()

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

In [30]:
food_data.head()

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


# Eliminating rating 3

In [31]:
food_data = food_data[food_data.Score != 3]

In [32]:
def f(r):
    if r > 3:
        return 1
    else:
        return 0
food_data['Sentiment'] = food_data.Score.map(f)

In [33]:
food_data.head()

,Score,Text,Sentiment
0,5,I have bought several of the Vitality canned d...,1
1,1,Product arrived labeled as Jumbo Salted Peanut...,0
2,4,This is a confection that has been around a fe...,1
3,2,If you are looking for the secret ingredient i...,0
4,5,Great taffy at a great price. There was a wid...,1


In [34]:
food_data.Sentiment.value_counts()

1    443777
0     82037
Name: Sentiment, dtype: int64

# Balancing the data

In [35]:
from sklearn.utils import resample

In [36]:
#for value in food_data'Sentiment'
  #  food_data_ = food_data.groupby('Sentiment')
food_data = food_data.groupby('Sentiment')

#print(food_data_g.groups)

for i,j in food_data:
    print(i)
    
    print(type(j))
    if i==0:
        food_data_minor = j
    if i==1:
        food_data_major = j
#print(food_data_minor)
#print(food_data_major)
food_data_major.head()

0
<class 'pandas.core.frame.DataFrame'>
1
<class 'pandas.core.frame.DataFrame'>


,Score,Text,Sentiment
0,5,I have bought several of the Vitality canned d...,1
2,4,This is a confection that has been around a fe...,1
4,5,Great taffy at a great price. There was a wid...,1
5,4,I got a wild hair for taffy and ordered this f...,1
6,5,This saltwater taffy had great flavors and was...,1


In [37]:
food_data_major_downsampled = resample(food_data_major,replace=False,n_samples=82037,random_state=123)
food_data_major_downsampled.Sentiment.value_counts()
food_data_balanced = pd.concat([food_data_minor,food_data_major_downsampled])
food_data_balanced.Sentiment.value_counts()

1    82037
0    82037
Name: Sentiment, dtype: int64

In [38]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split

In [39]:
food_data_balanced.head()

,Score,Text,Sentiment
1,1,Product arrived labeled as Jumbo Salted Peanut...,0
3,2,If you are looking for the secret ingredient i...,0
12,1,My cats have been happily eating Felidae Plati...,0
16,2,I love eating them and they are good for watch...,0
26,1,"The candy is just red , No flavor . Just plan...",0


In [40]:
food_data_balanced.drop(['Score'],axis = 1, inplace = True)
food_data_balanced.head()

,Text,Sentiment
1,Product arrived labeled as Jumbo Salted Peanut...,0
3,If you are looking for the secret ingredient i...,0
12,My cats have been happily eating Felidae Plati...,0
16,I love eating them and they are good for watch...,0
26,"The candy is just red , No flavor . Just plan...",0


In [41]:
X= food_data_balanced["Text"]
y = food_data_balanced["Sentiment"]

# Splitting data into test and training set

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [43]:
vect = CountVectorizer().fit(X_train)
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [44]:
X_train_vectorized = vect.transform(X_train)
X_train_vectorized

<114851x60157 sparse matrix of type '<class 'numpy.int64'>'
	with 6344648 stored elements in Compressed Sparse Row format>

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer().fit(X_train)
len(vect.get_feature_names())

60157

In [46]:
vect = CountVectorizer(ngram_range = (1,2)).fit(X_train)
X_train_vectorized = vect.transform(X_train)
len(vect.get_feature_names())

1258147

# Fitting the model and testing the accuracy

In [47]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_vectorized, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [48]:
from sklearn.metrics import roc_auc_score

predictions = model.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.9243011310146755


In [49]:
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression()
model1.fit(X_train_vectorized, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [50]:
from sklearn.metrics import roc_auc_score

predictions = model1.predict(vect.transform(X_test))

print('AUC: ', roc_auc_score(y_test, predictions))

AUC:  0.9428125046225588


# Testing the model

In [52]:
print(model1.predict(vect.transform(['The sweet does not  tastes good, I would never buy them again',
                                     'The sweet is not bad, I will buy them again'])))

[0 1]
